In [65]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import sys
import time
import copy
import string
import json
import pickle as pk

from collections import Counter

from math import sin, asin, cos, radians, fabs, sqrt

from sklearn.cluster import KMeans,MiniBatchKMeans,AgglomerativeClustering,SpectralClustering
from sklearn.datasets import make_blobs

############################    从文件中 读取 数据   ##############
def read_q(filename):
    f = open(filename,"rb+")
    st = pk.load(f)
    f.close()
    return st

##########################    向文件中  写 数据  #####################
def write_q(st, filename):
    f = open(filename, "wb+")
    pk.dump(st,f)
    f.close()


In [66]:
####################   proba()  把 staying list  转换为概率特征    ############
#  输入: staying list   一个POI上的停留时间list ;  把停留时间化为了9个时段0::8
#  输出: st_f      停留时间概率分布，  
def proba(st):
    t = st.copy()
    for i in range(0, len(st)):
    
        if 10 <= st[i] < 30:
            t[i] = 0
        elif 30 <= st[i] < 60:
            t[i] = 1
        elif 60 <= st[i] < 90:
            t[i] = 2
        elif 90 <= st[i] < 120:
            t[i] = 3
        elif 120 <= st[i] < 150:
            t[i] = 4
        elif 150 <= st[i] < 180:
            t[i] = 5
        elif 180 <= st[i] < 210:
            t[i] = 6
        elif 210 <= st[i] < 240:
            t[i] = 7
        else:
            t[i] = 8
    
    b = len(t)
    a = Counter(t)
    #print(a)
    stf = [0]*9        # staying list 转化为 fecture vector
    #print(st_f)

    for key, value in a.items():
        stf[key] = round(value/b, 3)
        
    return stf   

In [14]:
######################   mb_kmeans  聚类 分类   #####################
#输入 list_dic     [{},{},{}]  POI_list
#输出 l_l_d   list_list_dic  [[{},{},{}...],[{},{},{}...],[],[]]
def mb_kmeans(list_dic, n):
    

    print("调用 mb-kmeans，聚类前list的长度为： ", len(list_dic))
    l_l_d = []
    a = []
    for i in range(0, len(list_dic)):
        b = list_dic[i]["staying"]
        c = proba(b)
        a.append(c)
        

    X = np.array(a)
    print("参与聚类的list 的长度", len(X))
    y_pred = MiniBatchKMeans(n_clusters = n ).fit_predict(X)
    d = 0    
    for j in range(n):
            
        a1 = []     # 为每个聚类建立一个列表
        for k in range(0, len(X)):
                 
            if y_pred[k] == j:
                    
                a1.append(list_dic[k])
        print("聚类后，第一类list的长度为： ",len(a1))
        d += len(a1)
        l_l_d.append(a1)
        
    print("聚类后，两层列表总长度为： ", d)
    
    return l_l_d

In [67]:
######################   a_cluster  聚类 分类   #####################
#输入 list_dic     [{},{},{}]  POI_list
#输出 l_l_d   list_list_dic
def a_cluster(list_dic, n):
    

    print("调用 a_cluster，聚类前list的长度为： ", len(list_dic))
    l_l_d = []
    a = []
    for i in range(0, len(list_dic)):
        b = list_dic[i]["staying"]
        c = proba(b)
        a.append(c)
        

    X = np.array(a)
    print("参与聚类的list 的长度", len(X))
    y_pred = AgglomerativeClustering(n_clusters = n ).fit_predict(X)
    d = 0    
    for j in range(n):
            
        a1 = []
        for k in range(0, len(X)):
                 
            if y_pred[k] == j:
                    
                a1.append(list_dic[k])
        print("聚类后，第一类list的长度为： ",len(a1))
        d += len(a1)
        l_l_d.append(a1)
        
    print("聚类后，两层列表总长度为： ", d)
    
    return l_l_d



In [62]:
######################   s_cluster  聚类 分类   #####################
#输入 list_dic     [{},{},{}]  POI_list
#输出 l_l_d   list_list_dic
def s_cluster(list_dic, n):
    

    print("调用 s_cluster，聚类前list的长度为： ", len(list_dic))
    l_l_d = []
    a = []
    for i in range(0, len(list_dic)):
        b = list_dic[i]["staying"]
        c = proba(b)
        a.append(c)
        

    X = np.array(a)
    print("参与聚类的list 的长度", len(X))
    y_pred = SpectralClustering(n_clusters = n,affinity = "nearest_neighbors" ).fit_predict(X)
    #y_pred = spectral_clustering( ).fit_predict(X)
    d = 0    
    for j in range(n):
            
        a1 = []
        for k in range(0, len(X)):
                 
            if y_pred[k] == j:
                    
                a1.append(list_dic[k])
        print("聚类后，第一类list的长度为： ",len(a1))
        d += len(a1)
        l_l_d.append(a1)
        
    print("聚类后，两层列表总长度为： ", d)
    
    return l_l_d



In [68]:
########################  读取 st_poi_tree ########################
##   {"enter"}{"bar"}[{},{},{}]
filename = ".\\os_file\\st_poi_tree.pkl"        
st_poi_tree = read_q(filename)
st_poi_tree["enter"]["bar"]

[{'id': '大堂吧',
  'location': {'lat': 39.996445, 'lng': 116.312735},
  'staying': [77.04999993788078],
  'category': 'bar'},
 {'id': '京都信苑饭店-大堂酒吧',
  'location': {'lat': 39.904334999999996, 'lng': 116.31858600000001},
  'staying': [12.249999940395355],
  'category': 'bar'},
 {'id': '半山酒吧',
  'location': {'lat': 39.973151, 'lng': 116.48791299999999},
  'staying': [35.41666665812954, 151.58333333325572],
  'category': 'bar'},
 {'id': 'juice bar',
  'location': {'lat': 39.918248, 'lng': 116.31093600000001},
  'staying': [42.083333290647715, 151.28333336790092],
  'category': 'bar'},
 {'id': '酒吧',
  'location': {'lat': 39.975998, 'lng': 116.32942800000001},
  'staying': [14.19999997713603,
   20.000000012805685,
   20.016666578594595,
   14.400000002933666,
   10.516666610492393,
   12.366666707675904,
   46.233333349227905,
   20.39999990724027,
   14.499999932013452,
   14.466666674707085,
   109.61666668416001,
   90.96666666213423,
   141.46666675340384,
   31.41666661365889,
   48.0166

In [70]:
#####################  第一次a_cluster   ###############################
# n    聚类个数
filename = ".\\os_file\\st_poi_clusters\\a_cluster\\st_poi_cluster_1st.pkl"
n = 4
list_dic = []
st_poi_kmeans1 = copy.deepcopy(st_poi_tree)
a = 0
for item1 in st_poi_tree:
    print("第一大类 ", item1)
    b = 0
    
    for item2 in st_poi_tree[item1]:
       
        list_dic = st_poi_tree[item1][item2]
        print("    当前小类： ", item2, "POI长度为： ",len(list_dic))
        b += len(list_dic)
        
        if len(list_dic) < 10:
            continue
        
        k_m = a_cluster(list_dic, n)    
       
        st_poi_kmeans1[item1][item2] = k_m
    print("第一大类： ",item1,"共有POI：",b, "个")
    
    a += b    
print("共处理POI： ",a," 个")
write_q(st_poi_kmeans1, filename)

第一大类  enter
    当前小类：  bar POI长度为：  27
调用 a_cluster，聚类前list的长度为：  27
参与聚类的list 的长度 27
聚类后，第一类list的长度为：  5
聚类后，第一类list的长度为：  19
聚类后，第一类list的长度为：  2
聚类后，第一类list的长度为：  1
聚类后，两层列表总长度为：  27
    当前小类：  bath POI长度为：  124
调用 a_cluster，聚类前list的长度为：  124
参与聚类的list 的长度 124
聚类后，第一类list的长度为：  44
聚类后，第一类list的长度为：  10
聚类后，第一类list的长度为：  9
聚类后，第一类list的长度为：  61
聚类后，两层列表总长度为：  124
    当前小类：  bbulid POI长度为：  356
调用 a_cluster，聚类前list的长度为：  356
参与聚类的list 的长度 356
聚类后，第一类list的长度为：  135
聚类后，第一类list的长度为：  174
聚类后，第一类list的长度为：  32
聚类后，第一类list的长度为：  15
聚类后，两层列表总长度为：  356
    当前小类：  cinema POI长度为：  2
    当前小类：  farm POI长度为：  7
    当前小类：  ktv POI长度为：  38
调用 a_cluster，聚类前list的长度为：  38
参与聚类的list 的长度 38
聚类后，第一类list的长度为：  3
聚类后，第一类list的长度为：  28
聚类后，第一类list的长度为：  5
聚类后，第一类list的长度为：  2
聚类后，两层列表总长度为：  38
    当前小类：  scenic POI长度为：  515
调用 a_cluster，聚类前list的长度为：  515
参与聚类的list 的长度 515
聚类后，第一类list的长度为：  47
聚类后，第一类list的长度为：  188
聚类后，第一类list的长度为：  45
聚类后，第一类list的长度为：  235
聚类后，两层列表总长度为：  515
    当前小类：  tea POI长度为：  8
第一大类：  ent

In [71]:
#############################  2次  KMeans   ######################
#  输入 st_poi_kmeans1
#  输出 st_poi_kmeans2
filename = ".\\os_file\\st_poi_clusters\\a_cluster\\st_poi_cluster_2nd.pkl"
n = 4
#list_dict = []
st_poi_kmeans2 = copy.deepcopy(st_poi_kmeans1)
a = 0
for item1 in st_poi_kmeans1:
    print("第一大类： ",item1)
    
    a1= 0
    for item2 in st_poi_kmeans1[item1]:
        b = 0
        st = st_poi_kmeans1[item1][item2]
        print("    当前小类： ",item2)
        if 0 <= len(st)< 4 or 4 < len(st) < 10:  #未动过的点
            b += len(st)
            a1 += b
            continue
        
        if type(st[0]).__name__ == 'dict':  # len =4 的未动过的点  根据下一级对象类型
            b += len(st)
            a1 += b
            continue
        
        #type(st[0]).__name__ == 'list':
        
        for i in range(len(st)):    
            list_dict = st[i]
            b += len(list_dict)
            
            if len(list_dict) < 10:
                continue
            
            k_m = a_cluster(list_dict, n)
            
            st_poi_kmeans2[item1][item2][i] = k_m
        
        a1 += b
        print(item2,"共处理POI： ",b,"个")
    a += a1
    print(item1,"共处理POI： ",a1,"个")
print("第二次K-means 共处理POI：",a,"个")
            
write_q(st_poi_kmeans2,filename)

第一大类：  enter
    当前小类：  bar
调用 a_cluster，聚类前list的长度为：  19
参与聚类的list 的长度 19
聚类后，第一类list的长度为：  16
聚类后，第一类list的长度为：  1
聚类后，第一类list的长度为：  1
聚类后，第一类list的长度为：  1
聚类后，两层列表总长度为：  19
bar 共处理POI：  27 个
    当前小类：  bath
调用 a_cluster，聚类前list的长度为：  44
参与聚类的list 的长度 44
聚类后，第一类list的长度为：  14
聚类后，第一类list的长度为：  26
聚类后，第一类list的长度为：  2
聚类后，第一类list的长度为：  2
聚类后，两层列表总长度为：  44
调用 a_cluster，聚类前list的长度为：  10
参与聚类的list 的长度 10
聚类后，第一类list的长度为：  7
聚类后，第一类list的长度为：  1
聚类后，第一类list的长度为：  1
聚类后，第一类list的长度为：  1
聚类后，两层列表总长度为：  10
调用 a_cluster，聚类前list的长度为：  61
参与聚类的list 的长度 61
聚类后，第一类list的长度为：  58
聚类后，第一类list的长度为：  1
聚类后，第一类list的长度为：  1
聚类后，第一类list的长度为：  1
聚类后，两层列表总长度为：  61
bath 共处理POI：  124 个
    当前小类：  bbulid
调用 a_cluster，聚类前list的长度为：  135
参与聚类的list 的长度 135
聚类后，第一类list的长度为：  44
聚类后，第一类list的长度为：  76
聚类后，第一类list的长度为：  10
聚类后，第一类list的长度为：  5
聚类后，两层列表总长度为：  135
调用 a_cluster，聚类前list的长度为：  174
参与聚类的list 的长度 174
聚类后，第一类list的长度为：  171
聚类后，第一类list的长度为：  1
聚类后，第一类list的长度为：  1
聚类后，第一类list的长度为：  1
聚类后，两层列表总长度为：  174
调用 a_cluster，聚类